In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import nltk
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from nltk.stem import PorterStemmer
import re
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

In [5]:
df1 = pd.read_csv("Data/train.txt",header=None, sep=";", names=["Comment", "Emotion"], encoding="utf-8")
df2 = pd.read_csv("Data/text_emotion_data_2.csv")
df3 = pd.read_csv("Data/emotion_dataset_2.csv")
df4 = pd.read_csv("Data/Emotion_classify_Data.csv")
df5 = pd.read_csv("Data/Emotion(angry).csv")
df6 = pd.read_csv("Data/Emotion(happy).csv")
df7 = pd.read_csv("Data/Emotion(sad).csv")

In [6]:
df1 = df1.rename(columns = {"Comment":"Text"})
df2 = df2.rename(columns = {"content":"Text","sentiment":"Emotion"})
df2 = df2[["Text","Emotion"]]
df3 = df3[["Text","Emotion"]]
df4 = df4.rename(columns = {"Comment":"Text"})
df = pd.concat([df1,df2,df3,df4,df5,df6,df7])
df = df.drop_duplicates()
df["Emotion"] = df["Emotion"].replace({"sadness":"sad"})
df["Emotion"] = df["Emotion"].replace({"happiness":"joy","fun":"joy","relief":"joy","enthusiasm":"joy","happy":"joy"})
df["Emotion"] = df["Emotion"].replace({"happiness":"joy","fun":"joy","relief":"joy","enthusiasm":"joy","happy":"joy"})
df["Emotion"] = df["Emotion"].replace({"worry":"fear"})
df["Emotion"] = df["Emotion"].replace({"hate":"anger","disgust":"anger","angry":"anger"})
df["Emotion"] = df["Emotion"].replace({"empty":"neutral","boredom":"neutral","angry":"anger"})
df = df[df["Emotion"].isin(["joy","sad","fear","neutral"])]

In [7]:
df = df.reset_index(drop = True)

In [8]:
nltk.download('stopwords')

# Load the English stopwords
stopwords = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bnsla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def clean_text(docs):
    # stopwords = nltk.corpus.stopwords.words("English")
    docs = docs.lower()
    docs = re.sub("[^a-zA-Z]", " ", docs)
    docs = docs.split()
    stemmer = PorterStemmer()
    words = []
    for i in docs:
        if i not in stopwords:
            word = stemmer.stem(i)
            words.append(word)
    doc = " ".join(words)
    return doc

def length(doc):
    doclen = len(doc.split(" "))
    return doclen

In [10]:
df["cleaned_doc"] = df["Text"].apply(clean_text)

In [11]:
df["length"] = df["cleaned_doc"].apply(length)

In [12]:
df = df[df["length"] != 1009]

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["cleaned_doc"],)
vocab_size = len(tokenizer.word_index) + 1
input_vector = tokenizer.texts_to_sequences(df["cleaned_doc"])
input_vector = pad_sequences(sequences=input_vector, maxlen=300, padding='pre')

In [14]:
x_train = input_vector

In [15]:
lb = LabelEncoder()
y_train = lb.fit_transform(df["Emotion"])

In [16]:
df

,Text,Emotion,cleaned_doc,length
0,i didnt feel humiliated,sad,didnt feel humili,3
1,i can go from feeling so hopeless to so damned...,sad,go feel hopeless damn hope around someon care ...,9
2,ive been feeling a little burdened lately wasn...,sad,ive feel littl burden late wasnt sure,7
3,i feel as confused about life as a teenager or...,fear,feel confus life teenag jade year old man,8
4,i have been with petronas for years i feel tha...,joy,petrona year feel petrona perform well made hu...,9
...,...,...,...,...
67911,Stop crying over yesterday and start smiling f...,sad,stop cri yesterday start smile tomorrow,6
67912,An Eye with Dust ‘n A Heart with Trust Always ...,sad,eye dust n heart trust alway cri,7
67913,Tears come from the heart and not from the brain.,sad,tear come heart brain,4
67914,"Sometimes you have to hold your head up high, ...",sad,sometim hold head high blink away tear say goo...,10


In [17]:
y_train = to_categorical(y_train)

In [18]:
y_train

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [18]:
# Build and compile the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=150, input_length=300))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
callback = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
model.fit(x_train, y_train, epochs=30, batch_size=32, verbose=1, callbacks=[callback])

Epoch 1/30
2123/2123 [==============================] - 465s 218ms/step - loss: 1.0465 - accuracy: 0.5552
Epoch 2/30
2123/2123 [==============================] - 464s 219ms/step - loss: 0.7991 - accuracy: 0.6854
Epoch 3/30
2123/2123 [==============================] - 459s 216ms/step - loss: 0.5863 - accuracy: 0.7834
Epoch 4/30
2123/2123 [==============================] - 7283s 3s/step - loss: 0.4487 - accuracy: 0.8335
Epoch 5/30
2123/2123 [==============================] - 463s 218ms/step - loss: 0.3610 - accuracy: 0.8618
Epoch 6/30
2123/2123 [==============================] - 454s 214ms/step - loss: 0.2993 - accuracy: 0.8849
Epoch 7/30
2123/2123 [==============================] - 455s 214ms/step - loss: 0.2592 - accuracy: 0.8992
Epoch 8/30
2123/2123 [==============================] - 569s 268ms/step - loss: 0.2303 - accuracy: 0.9108
Epoch 9/30
2123/2123 [==============================] - 458s 216ms/step - loss: 0.2024 - accuracy: 0.9205
Epoch 10/30
2123/2123 [=========================

2123/2123 [==============================] - 556s 262ms/step - loss: 0.0549 - accuracy: 0.9794
Epoch 27/30
2123/2123 [==============================] - 564s 266ms/step - loss: 0.0520 - accuracy: 0.9805
Epoch 28/30
2123/2123 [==============================] - 2401s 1s/step - loss: 0.0508 - accuracy: 0.9806
Epoch 29/30
2123/2123 [==============================] - 540s 254ms/step - loss: 0.0497 - accuracy: 0.9815
Epoch 30/30
2123/2123 [==============================] - 568s 268ms/step - loss: 0.0490 - accuracy: 0.9822


In [30]:
model.save('model1.h5')

C:\Users\bnsla\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
import pickle
pickle.dump(lb,open("lb.pkl",'wb'))